<a href="https://colab.research.google.com/github/cod3astro/kaggle_ML_competition/blob/main/colab_kaggle_fertilizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
train_data = pd.read_csv('train.csv', index_col='id')
train_data.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
id,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,27,69,65,Sandy,Millets,30,6,18,28-28
2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,35,58,43,Red,Paddy,37,2,16,DAP


In [3]:
train_data.shape

(750000, 9)

In [4]:
train_data.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,31.503565,61.038912,45.184147,23.093808,9.478296,21.073227
std,4.025574,6.647695,11.794594,11.216125,5.765622,12.346831
min,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,28.000000,55.000000,35.000000,13.000000,4.000000,10.000000
50%,32.000000,61.000000,45.000000,23.000000,9.000000,21.000000
75%,35.000000,67.000000,55.000000,33.000000,14.000000,32.000000
max,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


In [5]:
train_data.isnull().any().any()

np.False_

In [6]:
for col in train_data.columns:
  unique_values = train_data[col].unique()
  print(f'{col} ({len(unique_values)} unique)')
  print(train_data[col].unique())

Temparature (14 unique)
[37 27 29 35 30 36 28 32 38 26 34 25 33 31]
Humidity (23 unique)
[70 69 63 62 58 59 51 50 61 60 57 72 68 55 52 64 71 56 66 54 65 67 53]
Moisture (41 unique)
[36 65 32 54 43 29 53 44 35 60 27 42 57 47 50 56 33 37 49 39 30 46 61 64
 26 34 51 48 28 38 52 62 55 40 25 59 31 45 58 41 63]
Soil Type (5 unique)
['Clayey' 'Sandy' 'Red' 'Loamy' 'Black']
Crop Type (11 unique)
['Sugarcane' 'Millets' 'Barley' 'Paddy' 'Pulses' 'Tobacco' 'Ground Nuts'
 'Maize' 'Cotton' 'Wheat' 'Oil seeds']
Nitrogen (39 unique)
[36 30 24 39 37 10 26 19 25 18 34 42 41 15  4  8  6 31 29 20 28 17 27  7
 12 14 35 33  9 16 40 21 23 32 22 11 38 13  5]
Potassium (20 unique)
[ 4  6 12  2  0 15 17  3 19 16  1 13  7  9 14  8 18 11  5 10]
Phosphorous (43 unique)
[ 5 18 16  4  9 22 35 29  2 12 19 39 30 32 34 40 28 33  0  7 13 24 21 27
 17 26  1 36 23 11 10 20 15 41  3 31  6 14 25 42  8 38 37]
Fertilizer Name (7 unique)
['28-28' '17-17-17' '10-26-26' 'DAP' '20-20' '14-35-14' 'Urea']


In [7]:
train_data.columns

Index(['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
      dtype='object')

In [8]:
train_data.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,31.503565,61.038912,45.184147,23.093808,9.478296,21.073227
std,4.025574,6.647695,11.794594,11.216125,5.765622,12.346831
min,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,28.000000,55.000000,35.000000,13.000000,4.000000,10.000000
50%,32.000000,61.000000,45.000000,23.000000,9.000000,21.000000
75%,35.000000,67.000000,55.000000,33.000000,14.000000,32.000000
max,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [10]:
features = ['Temparature','Nitrogen', 'Soil Type', 'Crop Type', 'Phosphorous', 'Potassium', 'Humidity', 'Moisture']
target = 'Fertilizer Name'
X = train_data[features]
y = train_data[target]
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
print(train_y.value_counts())

Fertilizer Name
14-35-14    91797
10-26-26    91046
17-17-17    89689
20-20       88879
28-28       88774
DAP         75712
Urea        74103
Name: count, dtype: int64


In [12]:
ordinal_cols = ['Crop Type', 'Soil Type']
numerical_cols = [ 'Nitrogen', 'Phosphorous', 'Potassium', 'Humidity', 'Moisture']

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_cols),
        ('num', 'passthrough', numerical_cols)
    ]
)

label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(eval_metric='mlogloss'))
    ]
)

model.fit(train_X, train_y_encoded)
y_preds_encoded = model.predict(val_X)
y_preds = label_encoder.inverse_transform(y_preds_encoded)

print("Accuracy:", accuracy_score(val_y, y_preds))
print("Classification Report:\n", classification_report(val_y, y_preds))
print("Confusion Matrix:\n", confusion_matrix(val_y, y_preds))

Accuracy: 0.1934
Classification Report:
               precision    recall  f1-score   support

    10-26-26       0.19      0.22      0.21     22841
    14-35-14       0.20      0.26      0.22     22639
    17-17-17       0.20      0.23      0.22     22764
       20-20       0.19      0.20      0.19     22010
       28-28       0.19      0.20      0.19     22384
         DAP       0.20      0.13      0.16     19148
        Urea       0.18      0.08      0.11     18214

    accuracy                           0.19    150000
   macro avg       0.19      0.19      0.19    150000
weighted avg       0.19      0.19      0.19    150000

Confusion Matrix:
 [[5024 4408 3750 3406 3511 1699 1043]
 [3729 5938 3680 3215 3311 1612 1154]
 [3711 4431 5220 3297 3433 1547 1125]
 [3560 4226 3495 4368 3452 1771 1138]
 [3594 4323 3621 3417 4487 1755 1187]
 [3195 3495 2988 2874 2993 2460 1143]
 [3022 3573 2971 2658 2933 1544 1513]]


In [15]:
test_data = pd.read_csv('test.csv')
test_X = test_data[features]
test_preds_encoded = model.predict(test_X)

test_preds = label_encoder.inverse_transform(test_preds_encoded)

submission = pd.DataFrame({
    'ID': test_data['id'],
    'Fertilizer Name': test_preds
})

submission.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>